# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import inauguralproject

# Question 1

BRIEFLY EXPLAIN HOW YOU SOLVE THE MODEL.

In [3]:
from scipy import optimize
# A. Define variables value
m = 1 #cash-on-hand
v = 10 #scale the disutility of labour
eps = 0.3 #Frish-elasticity
teta_0 = 0.4 #standard labour income tax
teta_1 = 0.1 #top bracket labour income tax
kappa = 0.4 #cut-off for the top labour income bracket
w = 1 #rate-wage
# b. define utility
def lab_sup(c,v,l,eps):
    return np.log(c) - v*(l**(1+1/eps))/(1+1/eps)

# c. define constraint
def cash_constr(m, teta_0, teta_1, w, l, kappa):
    return m + w*l - (teta_0*w*l + teta_1*max(w*l, 0))

# d. define objective function
def optimal_choice(m, l, w, eps, teta_0, teta_1, kappa, v):
    c = cash_constr(m, teta_0, teta_1, w, l, kappa)
    return - lab_sup(c,v,l,eps)
# e. call optimizer and calculate optimum
def opt_problem(m, w, eps, v, teta_0, teta_1, kappa):
    res = optimize.minimize_scalar(optimal_choice, method='bounded',bounds=(0,1),
    args=(w, eps, teta_0, teta_1, kappa, v, m))    
#extract and calculate optima for l,x,u
    optimal_l = res.x
    optimal_c = cash_constr(m, teta_0, teta_1, w, l = optimal_l, kappa = kappa)
    optimal_ut = lab_sup( c = optimal_c, v = v, l = optimal_l, eps = eps)
    return optimal_l, optimal_c, optimal_ut

#da

# Question 2

ADD ANSWER.

In [3]:
# code

# Question 3

ADD ANSWER.

In [4]:
# code

# Question 4

ADD ANSWER.

In [5]:
# code

# Question 5

ADD ANSWER.

In [6]:
# code

# Conclusion

ADD CONCISE CONLUSION.